# Import libraries and setup git root

In [1]:
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.cluster import KMeans
import torch
import numpy as np
import pandas as pd
import fastparquet ## make sure environment has fastparquet and NOT pyarrow
import itertools


In [2]:
import os

def find_repo_root(start_path):
    """
    useful general function for finding the (first, closest) repo root so github file paths work the same on different machines 
    """
    current_path = os.path.abspath(start_path)
    
    while True:
        # Check for the existence of the .git directory or other indicators
        if os.path.isdir(os.path.join(current_path, '.git')) or \
           os.path.isfile(os.path.join(current_path, 'README.md')):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        
        # Stop if we reach the root directory
        if parent_path == current_path:
            break
        
        current_path = parent_path

    return None  # Return None if not found

root = find_repo_root(os.getcwd())
root = root.replace('\\', '/')
print(root)


c:/Users/fitsl/Documents/Programming/UVM Programming Classes/PoCS/pocs_project


# Import the comments data

In [3]:
df = pd.read_parquet(f"{root}/Data/Whole_sets/merged_tropes_comments.parquet")

Now, we pick say 

In [4]:
df['reviews_extracted'].iloc[1]

["this fucking guy took a shower with socks on and it still wasn't the weirdest thing in the movie",
 'Maybe the greatest film of all time.',
 'God I forgot how absurdist and charming this is... Feels more like a series of sketches than a cohesive movie, but some of those,man, aregoddamn awesome. A stop motion hamburger rocking to Van Halen, dude skiing down a mountain withone ski, using a Hall and Oates song at the most opportune time. Besides, how can you hate a movie that ends with a kiss on top of a black camaro with John Cusack holding a saxophone?',
 '"gee, i\'m real sorry your mom blew up, ricky"',
 'So I finally got offered a great job yesterday and accepted it! To celebrate “operation: get life back on track” I decided to revisit a favorite!Life’s pretty good sometimes! :)',
 'We can just pretend Dan Schneider isn’t in this right',
 "One of the moreclever teen comediesof the 1980s. I love how absurd and cartoon-like this is from the get-go. The movie is filled to the brim with

In [ ]:
#### Schema idea from chatgpt #### 

# Example sets of actors, directors, and films
actors_list = ['Leonardo DiCaprio', 'Meryl Streep', 'Brad Pitt']  # You can expand this list
directors_list = ['Quentin Tarantino', 'Steven Spielberg', 'Martin Scorsese']  # Expand as needed
films_list = ['Inception', 'The Matrix', 'Pulp Fiction', 'Forrest Gump']  # Expand as needed

def replace_named_entities_with_keywords(text, actors, directors, films):
    # First, replace known actors, directors, and films with tags
    for actor in actors:
        text = re.sub(r'\b' + re.escape(actor) + r'\b', '[ACTOR]', text)
        
    for director in directors:
        text = re.sub(r'\b' + re.escape(director) + r'\b', '[DIRECTOR]', text)
        
    for film in films:
        text = re.sub(r'\b' + re.escape(film) + r'\b', '[FILM]', text)
    
    return text

def replace_named_entities_with_bert(text, nlp_ner):
    # Use BERT NER pipeline to extract named entities
    entities = nlp_ner(text)
    
    # Replace each recognized entity with a placeholder (except for those already tagged by the keyword pass)
    for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
        entity_text = text[entity['start']:entity['end']]
        
        # General NER replacement (skip already tagged entities like [ACTOR], [DIRECTOR], [FILM])
        if '[ACTOR]' not in entity_text and '[DIRECTOR]' not in entity_text and '[FILM]' not in entity_text:
            text = text[:entity['start']] + f"[{entity['entity']}]"+ text[entity['end']:]
    
    # Clean up tags by replacing 'B-' and 'I-' labels into a single tag
    text = re.sub(r'\[B-([A-Z]+)\](\s*\[I-\1\])+', r'[\1]', text)
    text = re.sub(r'\[[BI]-(\w+)\]', r'[\1]', text)

    return text

def process_review(text, nlp_ner):
    # Step 1: Keyword-based pass to tag actors, directors, and films
    text = replace_named_entities_with_keywords(text, actors_list, directors_list, films_list)
    
    # Step 2: Apply BERT NER pass to identify and tag other entities
    text = replace_named_entities_with_bert(text, nlp_ner)
    
    return text
